# 第8章の章末演習問題

※ ここではGoogle Colaraboratoryでの実行を想定しています。

※ Google Colaraboratoryでbashコマンドを実行するには、命令の前に!をつけます。

In [2]:
import torch
torch.manual_seed(0)

import datetime
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [5]:
%cd /content/drive/MyDrive/MyStudy/dlwpt-code-ja/p1ch8/

[Errno 2] No such file or directory: '/content/drive/MyDrive/MyStudy/dlwpt-code-ja/p1ch8/'
/content


### [1]nn.Conv2dコンストラクターに、kernel_size=5で5 × 5のサイズのカーネルを渡すようにモデルを変えてください。

In [14]:
# model
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8**2*8, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8**2*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)

        return out

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs+1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(datetime.datetime.now(), epoch, loss_train/len(train_loader)))

def validate(model, train_loader, val_loader):
    for name, loader in [('train', train_loader), ('val', val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted==labels).sum())

        print('acc {}: {:.2f}'.format(name, correct/total))

In [9]:
data_path = '../data-unversioned/p1ch7'

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ])
)

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ])
)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
label_map = {0: 0, 2: 1}
cifar2 = [(img, label_map[label]) for img, label, in cifar10 if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label, in cifar10_val if label in [0, 2]]

In [17]:
model = Net()
numel_list = [p.numel() for p in model.parameters()]
print(f'変更前のパラメータ数:{sum(numel_list)}')

model.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
model.conv2 = nn.Conv2d(16, 8, kernel_size=5, padding=2)
numel_list = [p.numel() for p in model.parameters()]
print(f'変更後のパラメータ数:{sum(numel_list)}')

変更前のパラメータ数:18090
変更後のパラメータ数:20906


#### a この変更により、モデル内のパラメーターにはどのような影響があるでしょうか。

In [ ]:
# 回答

#### b この変更は過剰適合を促進させてしまうでしょうか？それとも劣化させるでしょうか。

In [ ]:
# 回答

#### c https://pytorch.org/docs/stable/nn.html#conv2d. を読んでみましょう。

In [ ]:
# 省略

#### d kernel_size=(1,3) は何を行うでしょうか。説明してください。

In [ ]:
# 回答

#### e 上記のようなカーネルを備えたモデルはどのような振る舞いをするでしょうか。

In [ ]:
# 回答

### [2]鳥も飛行機も含まれていないが、モデルが95%以上の確信度で鳥も飛行機が写っていると主張するような画像を見つけることができますか？

#### a ニュートラルな画像を手動で編集して飛行機らしくすることはできますか？

In [ ]:
# 省略

#### b 飛行機の画像を手動で編集し、モデルを騙して鳥の報告をさせることはできますか？

In [ ]:
# 省略

#### c これらのタスクは、ネットワークの容量が少ないほど簡単になりますか？それとも、大きいほど簡単になりますか？

In [ ]:
# 回答